## 인터랙티브 Phi 3 Mini 4K 인스트럭트 챗봇과 Whisper

### 소개:
인터랙티브 Phi 3 Mini 4K 인스트럭트 챗봇은 사용자가 Microsoft Phi 3 Mini 4K 인스트럭트 데모와 텍스트 또는 음성 입력을 통해 상호작용할 수 있도록 하는 도구입니다. 이 챗봇은 번역, 날씨 업데이트, 일반 정보 수집 등 다양한 작업에 사용할 수 있습니다.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Huggingface 액세스 토큰 생성하기](https://huggingface.co/settings/tokens)

새 토큰 생성하기  
새 이름을 입력하세요  
쓰기 권한을 선택하세요  
토큰을 복사하여 안전한 장소에 저장하세요


다음은 Python 코드로, 두 가지 주요 작업을 수행합니다: `os` 모듈을 가져오고 환경 변수를 설정하는 것입니다.

1. `os` 모듈 가져오기:
   - Python의 `os` 모듈은 운영 체제와 상호작용할 수 있는 방법을 제공합니다. 이를 통해 환경 변수에 접근하거나, 파일 및 디렉토리 작업 등 다양한 운영 체제 관련 작업을 수행할 수 있습니다.
   - 이 코드에서는 `import` 문을 사용하여 `os` 모듈을 가져옵니다. 이 문은 현재 Python 스크립트에서 `os` 모듈의 기능을 사용할 수 있도록 합니다.

2. 환경 변수 설정:
   - 환경 변수는 운영 체제에서 실행 중인 프로그램이 접근할 수 있는 값입니다. 이는 여러 프로그램에서 사용할 수 있는 설정 정보나 기타 데이터를 저장하는 방법입니다.
   - 이 코드에서는 `os.environ` 딕셔너리를 사용하여 새로운 환경 변수를 설정합니다. 딕셔너리의 키는 `'HF_TOKEN'`이고, 값은 `HUGGINGFACE_TOKEN` 변수에서 할당됩니다.
   - `HUGGINGFACE_TOKEN` 변수는 이 코드 스니펫 바로 위에서 정의되며, `#@param` 구문을 사용하여 문자열 값 `"hf_**************"`이 할당됩니다. 이 구문은 주로 Jupyter 노트북에서 사용자 입력 및 매개변수 구성을 노트북 인터페이스에서 직접 설정할 수 있도록 사용됩니다.
   - `'HF_TOKEN'` 환경 변수를 설정함으로써, 프로그램의 다른 부분이나 동일한 운영 체제에서 실행 중인 다른 프로그램에서 이 값을 접근할 수 있습니다.

전체적으로 이 코드는 `os` 모듈을 가져오고, `HUGGINGFACE_TOKEN` 변수에 제공된 값을 사용하여 `'HF_TOKEN'`이라는 환경 변수를 설정합니다.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

이 코드 스니펫은 Jupyter Notebook 또는 IPython에서 현재 셀의 출력을 지우는 데 사용되는 clear_output이라는 함수를 정의합니다. 코드를 분석하고 그 기능을 이해해 봅시다:

clear_output 함수는 wait이라는 하나의 매개변수를 받습니다. 이 매개변수는 불리언 값이며, 기본값은 False로 설정되어 있습니다. 이 매개변수는 새로운 출력이 기존 출력을 대체할 준비가 될 때까지 기다릴지 여부를 결정합니다.

이 함수 자체는 현재 셀의 출력을 지우는 데 사용됩니다. Jupyter Notebook 또는 IPython에서 셀이 출력(예: 출력된 텍스트나 그래픽 플롯)을 생성하면, 해당 출력은 셀 아래에 표시됩니다. clear_output 함수는 이러한 출력을 지울 수 있도록 해줍니다.

코드 스니펫에서는 함수의 구현이 제공되지 않았으며, 이는 생략 기호(...)로 표시됩니다. 생략 기호는 출력을 지우는 실제 코드의 자리 표시자 역할을 합니다. 함수의 구현은 Jupyter Notebook 또는 IPython API와 상호작용하여 셀에서 기존 출력을 제거하는 작업을 포함할 수 있습니다.

전반적으로, 이 함수는 Jupyter Notebook 또는 IPython에서 현재 셀의 출력을 지우는 편리한 방법을 제공하며, 인터랙티브 코딩 세션 동안 표시된 출력을 관리하고 업데이트하기 쉽게 만들어줍니다.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Edge TTS 서비스를 사용하여 텍스트를 음성으로 변환(TTS)합니다. 관련 함수 구현을 하나씩 살펴보겠습니다:

1. `calculate_rate_string(input_value)`: 이 함수는 입력값을 받아 TTS 음성의 속도 문자열을 계산합니다. 입력값은 음성의 원하는 속도를 나타내며, 값이 1이면 정상 속도를 의미합니다. 함수는 입력값에서 1을 빼고 100을 곱한 후, 입력값이 1 이상인지 여부에 따라 부호를 결정합니다. 함수는 "{sign}{rate}" 형식으로 속도 문자열을 반환합니다.

2. `make_chunks(input_text, language)`: 이 함수는 입력 텍스트와 언어를 매개변수로 받아 언어별 규칙에 따라 텍스트를 청크로 나눕니다. 구현에서 언어가 "English"인 경우, 함수는 텍스트를 마침표(".")를 기준으로 나누고 앞뒤 공백을 제거합니다. 그런 다음 각 청크에 마침표를 추가하고 필터링된 청크 목록을 반환합니다.

3. `tts_file_name(text)`: 이 함수는 입력 텍스트를 기반으로 TTS 오디오 파일 이름을 생성합니다. 텍스트에서 끝에 있는 마침표를 제거하고(있을 경우), 텍스트를 소문자로 변환하며, 앞뒤 공백을 제거하고, 공백을 밑줄로 바꿉니다. 그런 다음 텍스트가 25자보다 길면 잘라내거나 비어 있으면 전체 텍스트를 사용합니다. 마지막으로 [`uuid`] 모듈을 사용하여 랜덤 문자열을 생성하고 잘린 텍스트와 결합하여 "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3" 형식의 파일 이름을 생성합니다.

4. `merge_audio_files(audio_paths, output_path)`: 이 함수는 여러 오디오 파일을 하나의 오디오 파일로 병합합니다. 오디오 파일 경로 목록과 출력 경로를 매개변수로 받습니다. 함수는 빈 `AudioSegment` 객체인 [`merged_audio`]를 초기화합니다. 그런 다음 각 오디오 파일 경로를 반복하며, `pydub` 라이브러리의 `AudioSegment.from_file()` 메서드를 사용하여 오디오 파일을 로드하고 현재 오디오 파일을 [`merged_audio`] 객체에 추가합니다. 마지막으로 병합된 오디오를 지정된 출력 경로에 MP3 형식으로 내보냅니다.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: 이 함수는 Edge TTS 서비스를 사용하여 TTS 작업을 수행합니다. 텍스트 청크 목록, 음성 속도, 음성 이름, 저장 경로를 매개변수로 받습니다. 청크 수가 1보다 많으면 개별 청크 오디오 파일을 저장할 디렉토리를 생성합니다. 그런 다음 각 청크를 반복하며, `calculate_rate_string()` 함수, 음성 이름, 청크 텍스트를 사용하여 Edge TTS 명령을 구성하고 `os.system()` 함수를 사용하여 명령을 실행합니다. 명령 실행이 성공하면 생성된 오디오 파일 경로를 목록에 추가합니다. 모든 청크를 처리한 후, `merge_audio_files()` 함수를 사용하여 개별 오디오 파일을 병합하고 지정된 저장 경로에 병합된 오디오를 저장합니다. 청크가 하나뿐인 경우, Edge TTS 명령을 직접 생성하고 오디오를 저장 경로에 저장합니다. 마지막으로 생성된 오디오 파일의 저장 경로를 반환합니다.

6. `random_audio_name_generate()`: 이 함수는 [`uuid`] 모듈을 사용하여 랜덤 오디오 파일 이름을 생성합니다. 랜덤 UUID를 생성하고 문자열로 변환한 후 처음 8자를 가져와 ".mp3" 확장자를 추가하여 랜덤 오디오 파일 이름을 반환합니다.

7. `talk(input_text)`: 이 함수는 TTS 작업을 수행하는 주요 진입점입니다. 입력 텍스트를 매개변수로 받습니다. 먼저 입력 텍스트의 길이를 확인하여 긴 문장(600자 이상)인지 판단합니다. 텍스트 길이와 `translate_text_flag` 변수 값을 기준으로 언어를 결정하고 `make_chunks()` 함수를 사용하여 텍스트 청크 목록을 생성합니다. 그런 다음 `random_audio_name_generate()` 함수를 사용하여 오디오 파일의 저장 경로를 생성합니다. 마지막으로 `edge_free_tts()` 함수를 호출하여 TTS 작업을 수행하고 생성된 오디오 파일의 저장 경로를 반환합니다.

이 함수들은 입력 텍스트를 청크로 나누고, 오디오 파일 이름을 생성하며, Edge TTS 서비스를 사용하여 TTS 작업을 수행하고, 개별 오디오 파일을 병합하여 하나의 오디오 파일로 만드는 작업을 함께 수행합니다.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

두 함수 convert_to_text와 run_text_prompt의 구현, 그리고 두 클래스 str과 Audio의 선언에 대한 설명입니다.

convert_to_text 함수는 audio_path를 입력으로 받아, whisper_model이라는 모델을 사용하여 오디오를 텍스트로 변환합니다. 이 함수는 먼저 gpu 플래그가 True인지 확인합니다. gpu 플래그가 True일 경우, whisper_model은 word_timestamps=True, fp16=True, language='English', task='translate'와 같은 특정 매개변수를 사용합니다. gpu 플래그가 False일 경우, whisper_model은 fp16=False로 사용됩니다. 변환된 텍스트는 'scan.txt'라는 파일에 저장되며, 텍스트로 반환됩니다.

run_text_prompt 함수는 message와 chat_history를 입력으로 받습니다. 이 함수는 phi_demo 함수를 사용하여 입력 메시지를 기반으로 챗봇의 응답을 생성합니다. 생성된 응답은 talk 함수에 전달되어 오디오 파일로 변환되며, 파일 경로가 반환됩니다. Audio 클래스는 오디오 파일을 표시하고 재생하는 데 사용됩니다. 오디오는 IPython.display 모듈의 display 함수를 사용하여 표시되며, Audio 객체는 autoplay=True 매개변수로 생성되어 오디오가 자동으로 재생됩니다. chat_history는 입력 메시지와 생성된 응답으로 업데이트되며, 빈 문자열과 업데이트된 chat_history가 반환됩니다.

str 클래스는 Python의 내장 클래스이며, 문자 시퀀스를 나타냅니다. 이 클래스는 문자열을 조작하고 작업할 수 있는 다양한 메서드를 제공합니다. 예를 들어, capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill 등이 있습니다. 이러한 메서드를 사용하면 검색, 대체, 포맷팅, 문자열 조작 등의 작업을 수행할 수 있습니다.

Audio 클래스는 사용자 정의 클래스이며, 오디오 객체를 나타냅니다. 이 클래스는 Jupyter Notebook 환경에서 오디오 플레이어를 생성하는 데 사용됩니다. 클래스는 data, filename, url, embed, rate, autoplay, normalize와 같은 다양한 매개변수를 허용합니다. data 매개변수는 numpy 배열, 샘플 리스트, 파일명 또는 URL을 나타내는 문자열, 또는 raw PCM 데이터를 포함할 수 있습니다. filename 매개변수는 오디오 데이터를 로드할 로컬 파일을 지정하는 데 사용되며, url 매개변수는 오디오 데이터를 다운로드할 URL을 지정하는 데 사용됩니다. embed 매개변수는 오디오 데이터를 데이터 URI로 임베드할지 또는 원본 소스를 참조할지를 결정합니다. rate 매개변수는 오디오 데이터의 샘플링 속도를 지정합니다. autoplay 매개변수는 오디오가 자동으로 재생될지 여부를 결정합니다. normalize 매개변수는 오디오 데이터를 최대 가능한 범위로 정규화(재조정)할지 여부를 지정합니다. Audio 클래스는 reload 메서드를 제공하여 파일 또는 URL에서 오디오 데이터를 다시 로드할 수 있으며, src_attr, autoplay_attr, element_id_attr와 같은 속성을 통해 HTML 오디오 요소의 해당 속성을 검색할 수 있습니다.

전반적으로, 이러한 함수와 클래스는 오디오를 텍스트로 변환하고, 챗봇의 오디오 응답을 생성하며, Jupyter Notebook 환경에서 오디오를 표시하고 재생하는 데 사용됩니다.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
